# Rasterize the Global Oceans and Seas v01 (2021-12-14) dataset

Flanders Marine Institute (2021). Global Oceans and Seas, version 1. Available online at https://www.marineregions.org/. https://doi.org/10.14284/542.

This dataset is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

[Disclaimer](https://www.marineregions.org/disclaimer.php)

> **Note: Run this notebook on the 8XL Analysis Sandbox**

In [1]:
from dotenv import load_dotenv

# Path to env file containing the waterbodies database credentials
# Only necessary on the Sandbox.
dotenv_path = "/home/jovyan/.env"
load_dotenv(dotenv_path=dotenv_path, verbose=True, override=True)

False

In [2]:
import logging
import os

import boto3
import geopandas as gpd
import rioxarray  # noqa F401
from datacube.utils.cog import to_cog
from odc.geo.geobox import GeoBox
from odc.geo.geom import Geometry
from odc.geo.xr import wrap_xr
from rasterio.features import rasterize
from waterbodies.grid import WaterbodiesGrid
from waterbodies.logs import logging_setup

In [3]:
verbose = 3
bucket = "deafrica-waterbodies-dev"
object_key = "waterbodies/v0.0.2/goas_v01/goas_v01.tif"

In [4]:
# Set up logging.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [5]:
gridspec = WaterbodiesGrid().gridspec

In [6]:
product_footprint = gpd.read_file("https://explorer.digitalearth.africa/api/footprint/wofs_ls_summary_alltime").to_crs(gridspec.crs)
product_footprint_geobox = GeoBox.from_geopolygon(geopolygon=Geometry(geom=product_footprint.geometry.iloc[0], crs=product_footprint.crs), resolution=gridspec.resolution, crs=gridspec.crs)

In [7]:
%%time
# Load the geopackage.
goas_v01_gdf = gpd.read_file("goas_v01.gpkg").to_crs(gridspec.crs)

[2024-04-24 19:57:51,896] {collection.py:243} ERROR - goas_v01.gpkg: No such file or directory


DriverError: goas_v01.gpkg: No such file or directory

In [8]:
%%time
# Clip to the product footprint
goas_v01_gdf_clipped = gpd.clip(goas_v01_gdf, product_footprint)

NameError: name 'goas_v01_gdf' is not defined

In [9]:
%%time
# Rasterize the clipped dataset.
goas_v01_gdf_clipped_np = rasterize(shapes=goas_v01_gdf_clipped.geometry, out_shape=product_footprint_geobox.shape, transform=product_footprint_geobox.transform)

NameError: name 'goas_v01_gdf_clipped' is not defined

In [10]:
%%time
goas_v01_gdf_clipped_xr = wrap_xr(im=goas_v01_gdf_clipped_np, gbox=product_footprint_geobox)

NameError: name 'goas_v01_gdf_clipped_np' is not defined

In [11]:
%%time
cog_bytes = to_cog(geo_im=goas_v01_gdf_clipped_xr)

NameError: name 'goas_v01_gdf_clipped_xr' is not defined

In [12]:
%%time
s3_client = boto3.client('s3')
s3_client.put_object(Body=cog_bytes,
                     Bucket=bucket,
                     Key=object_key)

[2024-04-24 19:57:52,393] {credentials.py:1180} INFO - Found credentials in environment variables.


NameError: name 'cog_bytes' is not defined